## PINECONE SETUP

In [40]:
%pip install sentence_tramsformer

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement sentence_tramsformer (from versions: none)
ERROR: No matching distribution found for sentence_tramsformer


In [8]:
%pip install pinecone

  Using cached pinecone-8.0.0-py3-none-any.whl.metadata (11 kB)
  Using cached pinecone_plugin_assistant-3.0.1-py3-none-any.whl.metadata (30 kB)
  Using cached pinecone_plugin_interface-0.0.7-py3-none-any.whl.metadata (1.2 kB)
  Using cached packaging-24.2-py3-none-any.whl.metadata (3.2 kB)
Using cached pinecone-8.0.0-py3-none-any.whl (745 kB)
Using cached pinecone_plugin_assistant-3.0.1-py3-none-any.whl (280 kB)
Using cached packaging-24.2-py3-none-any.whl (65 kB)
Using cached pinecone_plugin_interface-0.0.7-py3-none-any.whl (6.2 kB)

   ---------------------------------------- 0/4 [pinecone-plugin-interface]
  Attempting uninstall: packaging
   ---------------------------------------- 0/4 [pinecone-plugin-interface]
    Found existing installation: packaging 25.0
   ---------------------------------------- 0/4 [pinecone-plugin-interface]
    Uninstalling packaging-25.0:
   ---------------------------------------- 0/4 [pinecone-plugin-interface]
   ---------- -------------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-openai 0.3.35 requires langchain-core<1.0.0,>=0.3.78, but you have langchain-core 1.2.5 which is incompatible.


In [41]:
import os
from dotenv import load_dotenv
load_dotenv()
import tempfile, requests

API_KEY = os.getenv('PINECONE_API_KEY')

system_prompt = [
    'You are a document-grounded AI assistant.', 
    'You MUST answer user queries ONLY using the provided PDF text.', 
    'You are forbidden from using any external knowledge.', 
    'If the answer is missing, incomplete, or ambiguous in the document, you MUST say:', 
    '"I cannot answer this based on the uploaded document."', 
    'Never guess.', 
    'Never infer beyond the text.', 
    'Never add explanations not supported by the document.', 
    'Your goal is factual accuracy over completeness.'
]

In [24]:
from pinecone import Pinecone

pc = Pinecone(api_key = API_KEY)
INDEX_NAME = 'paperai'

if not pc.has_index(INDEX_NAME):
    pc.create_index_for_model(
        name = INDEX_NAME,
        cloud = 'aws',
        region = 'us-east-1',
        embed = {
            "model":"llama-text-embed-v2",
            "field_map":{"text": "text"}
        }
    )

index = pc.Index(INDEX_NAME)

In [34]:
records = []

for index, content in enumerate(system_prompt):
    records.append({
        '_id' : f'chunk_{index+1}',
        'chunk_text' : content
    })
    

In [39]:
with pc.Index(INDEX_NAME) as index:
    vectors = []
    for i, content in enumerate(system_prompt):
        # Manually get embedding using the index's embed function
        embedding = index.embed_text(content)["values"]  # returns list of floats

        vectors.append({
            "id": f"chunk_{i+1}",
            "values": embedding
        })

    # Upsert all vectors at once
    index.upsert(vectors=vectors)

print("All chunks successfully embedded and stored in Pinecone!")

AttributeError: 'Index' object has no attribute 'embed_text'